In [4]:
## Import the pip dependencies
import pandas as pd
from datetime import datetime

import sys
!{sys.executable} -m pip install azure-ai-evaluation

In [9]:
dataset = pd.read_csv("./slm_perf_output.csv")

sample = dataset.iloc[0]
print(sample)

Prompt      [{'role': 'system', 'content': 'You are a help...
Actual      {'tool_uses': [{'recipient_name': 'functions.s...
Expected    {'tool_uses': [{'recipient_name': 'functions.s...
Match                                                      No
Name: 0, dtype: object


In [14]:
import yaml
from azure.ai.evaluation import BleuScoreEvaluator, GleuScoreEvaluator, MeteorScoreEvaluator, RougeScoreEvaluator, RougeType

bleu = BleuScoreEvaluator()
glue = GleuScoreEvaluator()
meteor = MeteorScoreEvaluator(alpha = 0.9, beta = 3.0, gamma = 0.5)
rouge = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_L)


In [15]:
with open('./llama-fc_config.yaml') as f:
    d = yaml.load(f, Loader=yaml.FullLoader)
    
AZURE_SUBSCRIPTION_ID = d['config']['AZURE_SUBSCRIPTION_ID']
AZURE_RESOURCE_GROUP = d['config']['AZURE_RESOURCE_GROUP']
AZURE_WORKSPACE = d['config']['AZURE_WORKSPACE']
AZURE_DATA_NAME = d['config']['AZURE_SFT_DATA_NAME']    
DATA_DIR = d['config']['SFT_DATA_DIR']
CLOUD_DIR = d['config']['CLOUD_DIR']
HF_MODEL_NAME_OR_PATH = d['config']['HF_MODEL_NAME_OR_PATH']


In [20]:
df = pd.read_csv("./slm_perf_output.csv")
df.rename(columns = {"Actual" : "ground_truth", "Expected" : "response"}, inplace = True)
df.to_json("./eval_baseline_data.jsonl", orient = "records", lines = True)

In [24]:
! az ad user show --id priyakedia@ --query objectId  

In [ ]:
# <mySubscriptionID>: Subscription ID of the Azure AI Studio hub's linked storage account (available in Azure AI hub resource view in Azure Portal).
# <myResourceGroupName>: Resource group of the Azure AI Studio hub's linked storage account.
# <user-id>: User object ID for role assignment (retrieve with "az ad user show" command).

! az role assignment create --role "Storage Blob Data Contributor" --scope /subscriptions/8cebb108-a4d5-402b-a0c4-f7556126277f/resourceGroups/openairesourcegroup_priya --assignee-principal-type User --assignee-object-id "8cebb108-a4d5-402b-a0c4-f7556126277f"

In [22]:
from azure.ai.evaluation import evaluate

result = evaluate(
    data="./eval_baseline_data.jsonl",
    evaluators={
        "bleu": bleu,
        "gleu": glue,
        "meteor": meteor,
    },
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI Studio project
    azure_ai_project={
        "subscription_id": AZURE_SUBSCRIPTION_ID,
        "resource_group_name": "openAIResourceGroup_priya",
        "project_name": "priyakedia-1214",
    },
)

Prompt flow service has started...
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_bleu_bleu_asyncbleuscoreevaluator_1f6tw6hf_20241230_205033_749328
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_gleu_gleu_asyncgleuscoreevaluator_rqpbchlg_20241230_205033_760451
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_meteor_meteor_asyncmeteorscoreevaluator_cw89tpoq_20241230_205033_757892


EvaluationException: (UserError) Failed to upload evaluation run to the cloud due to insufficient permission to access the storage. Please ensure that the necessary access rights are granted.
Visit https://aka.ms/azsdk/python/evaluation/remotetracking/troubleshoot to troubleshoot this issue.

In [10]:
result = bleu(response=sample["Actual"], ground_truth=sample["Expected"])

print(result)

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


{'bleu_score': 0.8557309221341731}


In [11]:
from azure.ai.evaluation import GleuScoreEvaluator

gleu = GleuScoreEvaluator()